In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_selection import RFECV
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn import feature_selection
from sklearn import metrics
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split



In [3]:
def load_afl_data(pred_round):
    df_2017 = pd.read_csv("../data/afl_results_2017.csv")
    #print(df_2017.shape)
    df_2018 = pd.read_csv("../data/afl_results_2018.csv")
    #print(df_2018.shape)
    df_2019 = pd.read_csv("../data/afl_results_2019.csv")
    #print(df_2019.shape)
    df_2020 = pd.read_csv("../data/afl_results_2020.csv")
    #print(df_2020.shape)
    df_2021 = pd.read_csv("../data/afl_results_2021.csv")
    #print(df_2021.shape)
    df_2022 = pd.read_csv("../data/afl_results_2022.csv")
    pred_round_results = df_2022[df_2022['round.roundNumber'] == pred_round]
    df_2022 = df_2022[df_2022['round.roundNumber'] < pred_round]
    
    
    #print(df_2022.shape)
    df_all = pd.concat([df_2017, df_2018, df_2019, df_2020, df_2021,df_2022], axis=0)
    df_all['Date'] = pd.to_datetime(df_all['match.date']).dt.strftime("%Y-%m-%d")
    df_players_2017 = pd.read_csv("../data/afl_players_stats_2017.csv")
    #print(df_players_2017.shape)
    df_players_2018 = pd.read_csv("../data/afl_players_stats_2018.csv")
    #print(df_players_2018.shape)
    df_players_2019 = pd.read_csv("../data/afl_players_stats_2019.csv")
    #print(df_players_2019.shape)
    df_players_2020 = pd.read_csv("../data/afl_players_stats_2020.csv")
    #print(df_players_2020.shape)
    df_players_2021 = pd.read_csv("../data/afl_players_stats_2021.csv")
    #print(df_players_2021.shape)
    df_players_2022 = pd.read_csv("../data/afl_players_stats_2022.csv")
    df_players_2022 = df_players_2022[df_players_2022['Round'] < pred_round]
    
    #print(df_players_2022.shape)
    df_players = pd.concat([df_players_2017, df_players_2018, df_players_2019,df_players_2020,df_players_2021,df_players_2022], axis=0)
    #print(df_players.shape)
    #df_players.columns
    
    df_fixture = pd.read_csv("../data/fixture_2022.csv")
    df_next_games_teams = df_fixture[(df_fixture['round.roundNumber'] == pred_round)]
    df_next_games_teams = df_next_games_teams[['home.team.name','away.team.name','venue.name','compSeason.year','round.roundNumber']]
    df_next_games_teams = df_next_games_teams.rename(columns={'home.team.name': 'match.homeTeam.name', 'away.team.name': 'match.awayTeam.name','compSeason.year':'round.year'})
    df_next_games_teams['match.matchId'] = np.arange(len(df_next_games_teams))
    
    return df_all, df_players, df_fixture, df_next_games_teams, pred_round_results

def get_aggregate_player_stats(df=None):

    agg_stats = (df.rename(columns={ # Rename columns to lowercase
                    'Home.team': 'match.homeTeam.name',
                    'Away.team': 'match.awayTeam.name',
                    })
                   .groupby(by=['Date', 'Season', 'match.homeTeam.name', 'match.awayTeam.name'], as_index=False) # Groupby to aggregate the stats for each game
                   .sum()
                   #.drop(columns=['DE', 'TOG', 'Match_id']) # Drop columns
                   .assign(date=lambda df: pd.to_datetime(df.Date, format="%Y-%m-%d")) # Create a datetime object
                   .sort_values(by='Date')
                   .reset_index(drop=True))
    return agg_stats

In [4]:
df_all, df_players, df_fixture, df_next_games_teams, pred_round_results = load_afl_data(15)
agg_player = get_aggregate_player_stats(df_players)
#afl_df = df_all.merge(agg_player, on=['Date', 'match.homeTeam.name', 'match.awayTeam.name'], how='left')
#print(afl_df.shape)

In [15]:
df_players[['Home.team','Away.team']]

,Home.team,Away.team
0,Carlton,Richmond
1,Carlton,Richmond
2,Carlton,Richmond
3,Carlton,Richmond
4,Carlton,Richmond
...,...,...
5377,Gold Coast,Adelaide
5378,Gold Coast,Adelaide
5379,Gold Coast,Adelaide
5380,Gold Coast,Adelaide


In [16]:
df_all[['match.homeTeam.name','match.awayTeam.name']]

,match.homeTeam.name,match.awayTeam.name
0,Carlton,Richmond
1,Collingwood,Western Bulldogs
2,St Kilda,Melbourne
3,Sydney Swans,Port Adelaide
4,Essendon,Hawthorn
...,...,...
112,St Kilda,Essendon
113,Port Adelaide,Sydney Swans
114,West Coast Eagles,Geelong Cats
115,GWS Giants,Western Bulldogs
